In [81]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
from gl_functions import *
from spot_functions import *
from constants import timeout_sec, upload_filepath
from scipy.spatial.transform import Rotation as sciRot

## Learnings:

1. SE3 transformation multiplication is commutative:  

frame1_to_frame2 * frame2_to_frame3 =  frame2_to_frame3 * frame1_to_frame2 = frame1_to_frame3



In [3]:
#### CONSTANTS ####
look_at_locs = np.array([[1.1, 0, -0.5],[0, 1.1, -0.5]])

obj_pos_body = geometry_pb2.Vec3(x = 0.8, y = 0, z = -0.55)

arm_pos_bounds  = {'x': {'lower':  0.6, 'upper': 0.9},
                   'y': {'lower': -0.4, 'upper': 0.4},
                   'z': {'lower': -0.1, 'upper': 0.8}}

drop_rots = np.array([
    [1,0,0],
    [-1,0,0],
    [1,0,1],
    [-1,0,1],
    [1,0,-1],
    [-1,0,-1],
    [0,1,0],
    [0,1,1],
    [0,1,2],
    [0,1,3]
])

clear_waypoints = np.array([9, 10, 12, 25])

In [4]:
sdk, robot, id_client, \
robot_state_client, command_client, \
image_client, graph_nav_client, \
world_object_client, manipulation_api_client = init_robot('192.168.50.3')

In [5]:
lease_client, lease, lease_keep_alive = get_lease(robot)

In [86]:
os.environ.get("GROUNDLIGHT_API_TOKEN")

In [87]:
# Initialize Groundlight API
gl = Groundlight(api_token='api_2BdhzIXUiSKRCjMaB2xyYtOTxsm_5Xvr8eDFZVFEfRZq4GRk53iCXNrZ5UHZEN')
detectors = gl.list_detectors()
eraser_det = detectors.results[0]

In [7]:
rubiks_det = detectors.results[4]

In [63]:
get_batt_info(robot_state_client)

Battery at: 55.0 %. Robot has 3113 seconds left


(55.0, 3113)

In [62]:
success = power_on(robot)

In [64]:
bdcrc.blocking_selfright(command_client, timeout_sec = 20)

In [11]:
bdcrc.blocking_stand(command_client, timeout_sec = timeout_sec)

In [12]:
move_robot_relative(command_client, robot, 0, 0, 0)

In [13]:
def gripper_slow_open(command_client):
    
    gripper_angles = np.linspace(0, 1, 1000)
    
    for gripper_angle in gripper_angles:
        cmd = RobotCommandBuilder.claw_gripper_open_fraction_command(gripper_angle)
        cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)
        time.sleep(1/gripper_angles.shape[0])

In [14]:
def random_rot_drop(command_client, manipulation_api_client):
    drop_rot_ind = np.random.randint(drop_rots.shape[0])
    drop_rot = drop_rots[drop_rot_ind, :]
    
    rotObj = sciRot.from_euler('xyz',np.pi/2*drop_rot)
    rot_mtx = rotObj.as_matrix()
    rot_quat = bosdyn.client.math_helpers.Quat.from_matrix(rot_mtx)
    drop_pose_body_frame_proto = geometry_pb2.SE3Pose(
        position = geometry_pb2.Vec3(x = 0.8, y = 0, z = -0.3),  
        rotation = rot_quat.to_proto())
    drop_pose_body_frame_se3 = bosdyn.client.math_helpers.SE3Pose.from_proto(drop_pose_body_frame_proto)
    arm_cmd = RobotCommandBuilder.arm_pose_command(x = drop_pose_body_frame_se3.x,
                                               y = drop_pose_body_frame_se3.y,
                                               z = drop_pose_body_frame_se3.z,
                                               qw = drop_pose_body_frame_se3.rotation.w,
                                               qx = drop_pose_body_frame_se3.rotation.x,
                                               qy = drop_pose_body_frame_se3.rotation.y,
                                               qz = drop_pose_body_frame_se3.rotation.z,
                                               frame_name = 'body')

    body_cmd = RobotCommandBuilder.synchro_stand_command(body_height = -0.3)
    cmd = RobotCommandBuilder.build_synchro_command(arm_cmd, body_cmd)
    cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)
    bdcrc.block_until_arm_arrives(command_client, cmd_id, timeout_sec = time.time() + 3.0)
    time.sleep(0.5)
    
    gripper_slow_open(command_client)
    time.sleep(0.5)

    body_cmd = RobotCommandBuilder.synchro_stand_command(body_height = 0)
    cmd_id = command_client.robot_command(body_cmd, end_time_secs = time.time()+timeout_sec)
    time.sleep(1)

    success = blocking_stow(command_client, timeout_sec = 3)
    override_response = stowable_override(manipulation_api_client)
    time.sleep(2)

In [70]:
def rand_imgs_thru_det(command_client, image_client, gl, det):
    for j in range(len(look_at_locs)):
        look_at = look_at_locs[j]
        for i in range(3):
            #Move Arm to random position
            x_pos_rand = np.random.uniform(arm_pos_bounds['x']['lower'],arm_pos_bounds['x']['upper'])
            y_pos_rand = np.random.uniform(arm_pos_bounds['y']['lower'],arm_pos_bounds['y']['upper'])
            z_pos_rand = np.random.uniform(arm_pos_bounds['z']['lower'],arm_pos_bounds['z']['upper'])

            move_to = np.array([x_pos_rand, y_pos_rand, z_pos_rand])
            move_to_and_look_at(command_client, move_to, look_at, blocking = True)

            time.sleep(0.4)

            hand_color_img, img_response = cap_hand_image(image_client, source = 'hand_color_image')
            image_query = mat_thru_det(gl, det, hand_color_img)

In [18]:
def multiAttemptGrasp(*, manipulation_api_client, command_client, graph_nav_client, obj_pos_vision):
    for i in range(5):
        print(i)
        
        #grasp_params = manipulation_api_pb2.GraspParams(grasp_palm_to_fingertip = 1)
        grasp = manipulation_api_pb2.PickObject(frame_name = 'vision',
                                                object_rt_frame = obj_pos_vision)
        
        grasp_request = manipulation_api_pb2.ManipulationApiRequest(pick_object=grasp)
        response, success = make_grasp(grasp_request, manipulation_api_client, 
                                       command_client, graph_nav_client, verbose = True)

        curState = robot_state_client.get_robot_state()
        wrist_pos_rads = np.abs(curState.kinematic_state.joint_states[-1].position.value)

        if wrist_pos_rads < np.pi/36 or not success:
#            print(response, success)
            blocking_stow(command_client, timeout_sec = 15)
            time.sleep(3)
            continue
        else:
            return True

            
            print('GRIPPER IS CLOSED. FAILED TO GRAB CUBE')
        
    logging.warning('FAILED TO GRAB CUBE')
    return False

In [20]:
# def eulerToQuat():
    

In [21]:
def drop_cap_grasp(*, manipulation_api_client, command_client, image_client, robot_state_client, gl, det):
    random_rot_drop(command_client, manipulation_api_client)

    #get_a_tform_b returns transformation such that a_coords = tform * b_coords
    frame_tree_snapshot = robot.get_frame_tree_snapshot()
    body_to_vision = bosdyn.client.frame_helpers.get_a_tform_b(frame_tree_snapshot, frame_a = 'vision', frame_b ='body')
    obj_pos_body_se3_proto = geometry_pb2.SE3Pose(
        position = obj_pos_body,
        rotation = bosdyn.client.math_helpers.Quat(w = 0, 
                                               x = 0, 
                                               y = -(1/2)**0.5, 
                                               z = (1/2)**0.5).to_proto())
    obj_pos_body_se3 = bosdyn.client.math_helpers.SE3Pose.from_proto(obj_pos_body_se3_proto)
    obj_pos_vision_se3 = body_to_vision * obj_pos_body_se3

    move_robot_relative(command_client, robot, -0.3, 0, 0)

    #rand_imgs_thru_det(command_client, image_client, gl, rubiks_det)

    blocking_stow(command_client)
    
    #### EXPERIMENTAL
    
    body_cmd = RobotCommandBuilder.follow_arm_command()
    arm_cmd = RobotCommandBuilder.arm_pose_command(x = obj_pos_vision_se3.x,
                                               y = obj_pos_vision_se3.y,
                                               z = obj_pos_vision_se3.z + 0.5,
                                               qw = -(1/2)**0.5,
                                               qx = 0,
                                               qy = -(1/2)**0.5,
                                               qz = 0,
                                               frame_name = 'vision')
    cmd = RobotCommandBuilder.build_synchro_command(body_cmd, arm_cmd)
    cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)
    
    bdcrc.block_until_arm_arrives(command_client, cmd_id, timeout_sec = time.time() + 3.0)
    
    #EXXPERIMENAL END
    
    

    multiAttemptGrasp(manipulation_api_client = manipulation_api_client,
                      command_client = command_client,
                      graph_nav_client = graph_nav_client,
                      obj_pos_vision = obj_pos_vision_se3.position)

    cmd = RobotCommandBuilder.arm_stow_command()
    cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)

In [50]:
    random_rot_drop(command_client, manipulation_api_client)

    #get_a_tform_b returns transformation such that a_coords = tform * b_coords
    frame_tree_snapshot = robot.get_frame_tree_snapshot()
    body_to_vision = bosdyn.client.frame_helpers.get_a_tform_b(frame_tree_snapshot, frame_a = 'vision', frame_b ='body')
    obj_pos_body_se3_proto = geometry_pb2.SE3Pose(
        position = obj_pos_body,
        rotation = bosdyn.client.math_helpers.Quat(w = 0, 
                                               x = 0, 
                                               y = -(1/2)**0.5, 
                                               z = (1/2)**0.5).to_proto())
    obj_pos_body_se3 = bosdyn.client.math_helpers.SE3Pose.from_proto(obj_pos_body_se3_proto)
    obj_pos_vision_se3 = body_to_vision * obj_pos_body_se3


In [51]:
    move_robot_relative(command_client, robot, -0.3, 0, 0)



In [78]:
    #rand_imgs_thru_det(command_client, image_client, gl, rubiks_det)

    blocking_stow(command_client)

True

In [79]:
gripper_open(command_client)

In [80]:
gripper_close(command_client)

In [41]:
wp_loc_proto = graph_del_get_graph_loc(graph_nav_client)

In [86]:
    drop_cap_grasp(manipulation_api_client = manipulation_api_client,
               command_client = command_client,
               image_client = image_client,
               robot_state_client = robot_state_client,
               gl = gl,
               det = rubiks_det)

0
1
2


KeyboardInterrupt: 

In [34]:
for i in range(4):
    drop_cap_grasp(manipulation_api_client = manipulation_api_client,
               command_client = command_client,
               image_client = image_client,
               robot_state_client = robot_state_client,
               gl = gl,
               det = rubiks_det)

    upload_graph_and_snapshots(graph_nav_client, upload_filepath)
    success = graph_localize_fiducial(command_client, graph_nav_client, robot_state_client, robot)
    waypoint_list = create_waypoint_list(graph_nav_client)
    
    #wp_ind = np.random.choice(clear_waypoints)
    wp_ind = clear_waypoints[i]
    print(wp_ind)
    nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[wp_ind], cmd_duration = 15)
    wait_until_finished_nav(graph_nav_client, nav_to_cmd_id)
    
    wp_loc_proto = graph_del_get_graph_loc(graph_nav_client)

0


9


bosdyn.api.RetainLeaseResponse (LeaseUseError): 
    (resuming check-in)


0


10
0
1
2
3
4


12


KeyboardInterrupt: 

In [92]:
upload_graph_and_snapshots(graph_nav_client, upload_filepath)
success = graph_localize_fiducial(command_client, graph_nav_client, robot_state_client, robot)
waypoint_list = create_waypoint_list(graph_nav_client)

In [94]:
nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[0], cmd_duration = 15)

In [93]:
wp_ind_list = [0, 8, 10, 12, 15]
for wp_ind in wp_ind_list:
    blocking_stow(command_client)
    nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[wp_ind], cmd_duration = 15)
    wait_until_finished_nav(graph_nav_client, nav_to_cmd_id)
    prev_graph_state = graph_del_save_state(graph_nav_client, robot)
    gripper_open(command_client)
    rand_imgs_thru_det(command_client, image_client, gl, rubiks_det)
    gripper_close(command_client)
    upload_graph_and_snapshots(graph_nav_client, upload_filepath)
    response = graph_localize_saved_state(graph_nav_client, robot, prev_graph_state)

KeyboardInterrupt: 

In [66]:
move_to_and_look_at(command_client, [0.8,0,0.5], [1.2, 0, -0.5])


In [67]:
hand_color_image = cap_hand_image(image_client, 'hand_color_image')

KeyboardInterrupt: 

In [42]:
prev_graph_state = graph_del_save_state(graph_nav_client, robot)

In [43]:
move_robot_relative(command_client, robot, 1, -0.5, 0)

In [45]:
response = upload_graph_and_snapshots(graph_nav_client, upload_filepath)

In [47]:
response = graph_localize_saved_state(graph_nav_client, robot, prev_graph_state)

In [174]:
prev_loc_response = graph_nav_client.get_localization_state()
prev_graph_loc_proto = prev_loc_response.localization

prev_waypoint_id = prev_graph_loc_proto.waypoint_id

prev_waypoint_to_body_proto = prev_graph_loc_proto.waypoint_tform_body
prev_waypoint_to_body_se3 = bosdyn.client.math_helpers.SE3Pose.from_proto(prev_waypoint_to_body_proto)

frame_tree_snapshot = robot.get_frame_tree_snapshot()
prev_body_to_vision_se3 = bosdyn.client.frame_helpers.get_a_tform_b(frame_tree_snapshot, frame_a = 'vision', frame_b = 'body')

vision_to_waypoint_se3 = (prev_waypoint_to_body_se3 * prev_body_to_vision_se3).inverse()

In [175]:
clear_response = graph_nav_client.clear_graph()

In [176]:
move_robot_relative(command_client, robot, 1, -0.5, 0)

In [203]:
cmd = RobotCommandBuilder.synchro_stand_command(body_height = -0.0)
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)

In [194]:
frame_tree_snapshot = robot.get_frame_tree_snapshot()

#get_a_tform_b returns transformation such that a_coords = tform * b_coords
body_to_vision_se3 = get_a_tform_b(frame_tree_snapshot, frame_a = 'vision', frame_b = 'body')
body_wrt_waypoint_proto = (body_to_vision_se3 * vision_to_waypoint_se3).to_proto()

In [58]:
response = upload_graph_and_snapshots(graph_nav_client, upload_filepath)

In [213]:
fid_init_code_NO_FID = graph_nav_pb2.SetLocalizationRequest.FiducialInit.FIDUCIAL_INIT_NO_FIDUCIAL

In [215]:
init_guess_loc = nav_pb2.Localization(waypoint_id = prev_waypoint_id,
                                      waypoint_tform_body = body_wrt_waypoint_se3)

In [217]:
response = graph_nav_client.set_localization(initial_guess_localization=init_guess_loc,
                                  fiducial_init = fid_init_code_NO_FID)

In [147]:
graph_loc_proto

waypoint_id: "east-borzoi-Qy+f458Vx4G9V2azQGXfvQ=="
waypoint_tform_body {
  position {
    x: -0.03689809303772129
    y: 0.008745399235789625
    z: 0.014877651766474487
  }
  rotation {
    x: 0.003887769004423273
    y: -0.003006873615987585
    z: 0.0024820523091867905
    w: 0.9999804523052581
  }
}
timestamp {
  seconds: 1657238434
  nanos: 465764475
}
seed_tform_body {
  position {
    x: 2.1947273869739923
    y: -4.1665980464764
    z: 0.36236126821044307
  }
  rotation {
    x: -0.0004398306078047513
    y: 3.494274975483513e-05
    z: -0.6608845824278025
    w: 0.7504815241196132
  }
}

In [47]:
cmd = RobotCommandBuilder.arm_stow_command()
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)

In [36]:
    nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[25], cmd_duration = 15)
    wait_until_finished_nav(graph_nav_client, nav_to_cmd_id)

In [114]:
success = multiAttemptGrasp(manipulation_api_client = manipulation_api_client, 
                            command_client = command_client, 
                            graph_nav_client = graph_nav_client, 
                            obj_pos_vision = obj_pos_vision_se3.position)

NameError: name 'obj_pos_vision_se3' is not defined

In [81]:
multiAttemptGrasp(manipulation_api_client, command_client, graph_nav_client)

0


TypeError: Parameter to MergeFrom() must be instance of same class: expected bosdyn.api.Vec3 got str.

In [ ]:
move()

In [86]:
logging.getLogger().setLevel(logging.WARNING)

In [46]:
nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[8], cmd_duration = 15)

In [26]:
move_to = np.array([0.8, 0,-0.4])
look_at = np.array([0.8, 0, -3])

In [27]:
move_to_and_look_at(command_client, move_to, look_at)

In [90]:
def get_walking_params(max_linear_vel, max_rotation_vel):
    max_vel_linear = geometry_pb2.Vec2(x=max_linear_vel, y=max_linear_vel)
    max_vel_se2 = geometry_pb2.SE2Velocity(linear=max_vel_linear,
                                           angular=max_rotation_vel)
    vel_limit = geometry_pb2.SE2VelocityLimit(max_vel=max_vel_se2)
    params = RobotCommandBuilder.mobility_params()
    params.vel_limit.CopyFrom(vel_limit)
    return params

In [130]:
params = get_walking_params(2,1.55)

In [183]:
cmd = RobotCommandBuilder.battery_change_pose_command()
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)

In [19]:
move_robot_relative(command_client, robot, 1, 0, 0)

In [17]:
move_robot_relative

<function spot_functions.move_robot_relative(command_client, robot, x_meters, y_meters, theta_rad, timeout_sec=10, blocking=True)>